## db updator : ec2 esg db에 Product 테이블 생성 및 삽입
- Company 테이블 불러와서 Product 데이터프레임과 left join 후 esg db에 Product table  밀어 넣기 

In [1]:
# Company 테이블 불러오기
from table_selector import selectdb2df
company=selectdb2df('Company','./esg_ec2_config','.')



DB 연결 시작 !
DB 접속 성공
Company select 성공!


In [2]:
# product (녹색 및 위해제품 보유 기업 현황(12건, 녹색인증 : 2020인증기업->2021지속) 데이터프레임과 left join)

import pandas as pd
grn_hzd_df=pd.read_csv(r'C:\Users\Choi\workspace\sesac\1212~final_project_ESG\0.DATA\녹색,위해, 기업정보결합\kosdaq_report_green_hazard_ver5.csv',encoding='cp949')
grn_hzd_df

,CmpName,PctType,Year
0,디와이씨,G,2020
1,디와이씨,G,2021
2,우리산업,G,2020
3,우리산업,G,2021
4,동원개발,G,2020
5,동원개발,G,2021
6,대원산업,G,2020
7,대원산업,G,2021
8,CJ프레시웨이,H,2020
9,CJ ENM,H,2020


In [3]:
prt_cmp_merge=pd.merge(grn_hzd_df,company, how='left',left_on='CmpName', right_on='CmpName')
prt_cmp_merge

,CmpName,PctType,Year,ID,StockCode,Owner,BizNo,EstDate,Address,Industry,Product
0,디와이씨,G,2020,211,310870,"이종훈, 김용덕(각자대표이사)",5878101164,2018-10-22,경상북도 경산시 진량읍 공단7로 42 대원리 498번지 디와이씨 주식회사,제조업,자동차 부품
1,디와이씨,G,2021,211,310870,"이종훈, 김용덕(각자대표이사)",5878101164,2018-10-22,경상북도 경산시 진량읍 공단7로 42 대원리 498번지 디와이씨 주식회사,제조업,자동차 부품
2,우리산업,G,2020,381,215360,김정우,4168602172,2015-04-01,경기도 용인시 기흥구 지삼로 89,제조업,ACTUATOR 등
3,우리산업,G,2021,381,215360,김정우,4168602172,2015-04-01,경기도 용인시 기흥구 지삼로 89,제조업,ACTUATOR 등
4,동원개발,G,2020,1361,13120,"장복만, 이성휘 (각자 대표이사)",6048119275,1978-03-31,"부산광역시 수영구 수영로 754 동원빌딩 (민락동,센텀비스타동원)",건설업,도급공사
5,동원개발,G,2021,1361,13120,"장복만, 이성휘 (각자 대표이사)",6048119275,1978-03-31,"부산광역시 수영구 수영로 754 동원빌딩 (민락동,센텀비스타동원)",건설업,도급공사
6,대원산업,G,2020,1260,5710,"허재건, 허재명, 김재덕",1348104254,1968-09-10,경기도 안산시 단원구 원시로 179,제조업,자동차시트
7,대원산업,G,2021,1260,5710,"허재건, 허재명, 김재덕",1348104254,1968-09-10,경기도 안산시 단원구 원시로 179,제조업,자동차시트
8,CJ프레시웨이,H,2020,1112,51500,정성필,6038111270,1988-10-27,경기도 용인시 기흥구 기곡로 32,도매 및 소매업,"식자재유통, 단체급식"
9,CJ ENM,H,2020,1304,35760,"Ho Sung Kang, Sang Hyun Yoon",1068151510,1994-12-16,서울특별시 서초구 과천대로 870-13,정보통신업,"의류, 생활주방, 가전제품, 보석, 기타"


In [4]:
# 필요한 컬럼 선택
product_tb=prt_cmp_merge[['ID','Year','PctType']]
product_tb=product_tb.rename(columns={'ID':'CmpID'})
product_tb

,CmpID,Year,PctType
0,211,2020,G
1,211,2021,G
2,381,2020,G
3,381,2021,G
4,1361,2020,G
5,1361,2021,G
6,1260,2020,G
7,1260,2021,G
8,1112,2020,H
9,1304,2020,H


In [12]:
# EC2 DB의 Product 테이블 밀어넣기
# Product테이블에 데이터 삽입.

import pandas as pd

product_list=product_tb.values.tolist() #df를 list 로 만들기-> tuple로 만들기

data = list([tuple(x) for x in product_list])

import pymysql


# esg_ec2_config 파일 읽어오기
db_config = {}
with open('./esg_ec2_config', 'r') as f:
    for l in f.readlines():
        key, value = l.rstrip().split('=')
        db_config[key] = value


print('DB 연결 시작 !')
try:
    connect=pymysql.connect(**db_config
    )
    print('DB 접속 성공')
    #테이블 생성
    # create table PK설정주의.
    try:
        with connect.cursor() as cursor:
            sql = '''CREATE TABLE IF NOT EXISTS Product (
            ID int NOT NULL AUTO_INCREMENT PRIMARY KEY,
            CmpID int NOT NULL,
            Year varchar(8) NOT NULL,
            PctType varchar(8) NOT NULL
            )
            '''
            cursor.execute(sql)
            connect.commit()


    except:
        print('테이블 생성 실패')
    try:
        with connect.cursor() as cursor:

            sql = "insert into Product (CmpID,Year,PctType) values (%s,%s,%s)" #       
            cursor.executemany(sql,data) # executemany는 리스트를 받아옴      

            connect.commit()
            cursor.close() #db연결해제
            print('db insert 성공!')

    except Exception as e:
        print(e)


except Exception as e:

    print(e)
    print('DB 접속 실패')


DB 연결 시작 !
DB 접속 성공
db insert 성공!
